# [Edit Distance](https://leetcode.com/problems/edit-distance/)
## Description
Compute the minimum number of operations required to convert one word to another using insert, delete, or replace.
## Strategy
Use 2D DP table where `dp[i][j]` = edit distance between first `i` and `j` chars.

# Recursive Edit Distance Algorithm Explained

This recursive solution perfectly captures the **essence of the edit distance problem** through a top-down approach. Let me break down how it works:

## Core Logic

The function works **backwards** from the end of both strings, making decisions at each position.

### Base Cases
```typescript
if (word1Index == 0) return word2Index;  // Insert all remaining chars from word2
if (word2Index == 0) return word1Index;  // Delete all remaining chars from word1
```

**Intuition:** If one string is exhausted, we need operations equal to the remaining length of the other string.

### Recursive Cases

**1. Characters Match:**
```typescript
if (word1.charAt(word1Index - 1) == word2.charAt(word2Index - 1)) {
    return minDistanceRecur(word1, word2, word1Index - 1, word2Index - 1);
}
```
No operation needed - just move both pointers back.

**2. Characters Don't Match - Try All 3 Operations:**
```typescript
let insertOperation = minDistanceRecur(word1, word2, word1Index, word2Index - 1);
let deleteOperation = minDistanceRecur(word1, word2, word1Index - 1, word2Index);  
let replaceOperation = minDistanceRecur(word1, word2, word1Index - 1, word2Index - 1);

return Math.min(insertOperation, deleteOperation, replaceOperation) + 1;
```

## What Each Operation Means

### Insert Operation
- `(word1Index, word2Index - 1)`
- "Insert `word2[word2Index-1]` into `word1` at current position"
- Now we need to match `word1[0..word1Index-1]` with `word2[0..word2Index-2]`

### Delete Operation
- `(word1Index - 1, word2Index)`
- "Delete `word1[word1Index-1]` from `word1`"
- Now we need to match `word1[0..word1Index-2]` with `word2[0..word2Index-1]`

### Replace Operation
- `(word1Index - 1, word2Index - 1)`
- "Replace `word1[word1Index-1]` with `word2[word2Index-1]`"
- Now we need to match `word1[0..word1Index-2]` with `word2[0..word2Index-2]`

## Example Trace: "CAT" → "DOG"

```
minDistanceRecur("CAT", "DOG", 3, 3)
├── 'T' ≠ 'G', so try all 3 operations:
    ├── Insert 'G': minDistanceRecur("CAT", "DOG", 3, 2) + 1
    ├── Delete 'T': minDistanceRecur("CAT", "DOG", 2, 3) + 1  
    └── Replace 'T'→'G': minDistanceRecur("CAT", "DOG", 2, 2) + 1
```

## Recursive Tree Structure

```
                    ("CAT", "DOG", 3, 3)
                   /          |          \
            (3,2)+1      (2,3)+1      (2,2)+1
           /   |   \    /   |   \    /   |   \
       (3,1) (2,2) (2,1) (2,2) (1,3) (1,2) (1,2) (1,1)
         |     |     |     |     |     |     |     |
        ...   ...   ...   ...   ...   ...   ...   ...
```

## Time Complexity Issues

**Problem:** This has **exponential time complexity O(3^max(m,n))** due to:
- Each call makes up to 3 recursive calls
- Same subproblems solved multiple times

**Example of redundancy:**
- `(2,2)` appears multiple times in the tree
- `(1,2)` also appears multiple times

## Why It's Still Valuable

1. **Conceptual Clarity:** Shows the pure decision-making process
2. **Easy to Understand:** Maps directly to the problem description
3. **Foundation for DP:** This becomes the DP solution with memoization

## Converting to Efficient Solution

**Add Memoization:**
```typescript
const memo = new Map<string, number>();
const key = `${word1Index},${word2Index}`;
if (memo.has(key)) return memo.get(key);
// ... compute result ...
memo.set(key, result);
return result;
```


In [18]:
function minDistance(word1: string, word2: string): number {
    let word1Length = word1.length;
    let word2Length = word2.length;

    // Create 2D DP table
    let dp: number[][] = new Array(word1Length + 1).fill().map(() => new Array(word2Length + 1));
    
    // Base cases: First column - transforming word1 prefixes to empty string
    for(let i = 0; i <= word1Length; i++) {
        dp[i][0] = i; // Delete i characters
    }

    // Base cases: First row - transforming empty string to word2 prefixes
    for(let j = 0; j <= word2Length; j++) {
        dp[0][j] = j; // Insert j characters
    }
    
    // Fill the DP table
    for(let i = 1; i <= word1Length; i++) {
        for (let j = 1; j <= word2Length; j++) {
            if (word1[i - 1] === word2[j - 1]) {
                // Characters match, no operation needed
                dp[i][j] = dp[i - 1][j - 1];
            } else {
                // Characters don't match, try all three operations
                dp[i][j] = 1 + Math.min(
                    dp[i - 1][j],     // Delete from word1
                    dp[i][j - 1],     // Insert into word1
                    dp[i - 1][j - 1]  // Replace in word1
                );
            }
        }
    }

    return dp[word1Length][word2Length];
}

In [19]:
import { assertEquals } from "https://deno.land/std/testing/asserts.ts";

Deno.test("Edit Distance - Basic", () => {
  assertEquals(minDistance("horse", "ros"), 3);
});

Deno.test("Edit Distance - Same", () => {
  assertEquals(minDistance("abc", "abc"), 0);
});

Deno.test("Edit Distance - Insert All", () => {
  assertEquals(minDistance("", "abc"), 3);
});

Deno.test("Edit Distance - Real World", () => {
  assertEquals(minDistance("sunday", "saturday"), 3);
});



Edit Distance - Basic ... ok (0ms)
Edit Distance - Same ... ok (0ms)
Edit Distance - Insert All ... ok (0ms)
Edit Distance - Real World ... ok (0ms)

ok | 4 passed | 0 failed (0ms)
